In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
sns.set()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")  
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
def bar_chart(feature):
    survived = train_df[train_df['Survived']==1][feature].value_counts()
    dead = train_df[train_df['Survived']==0][feature].value_counts()
    df = pd.DataFrame([survived,dead])
    df.index = ['Survived','Dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

categorial = ['Sex', 'Embarked']
categorial_digit = ['Pclass' ]
nimeric = ['Age', 'SibSp', 'Parch', 'Fare']

for cat in (categorial+ categorial_digit):
    bar_chart(cat)

In [ ]:
def plotFacet(feature):
    facet = sns.FacetGrid(train_df, hue="Survived",aspect=4)
    facet.map(sns.kdeplot,feature,shade= True)
    facet.set(xlim=(0, train_df[feature].max()))
    facet.add_legend()
    plt.xlim(0)


for num in (nimeric):
    plotFacet(num)

In [ ]:
train_df.columns

In [ ]:
all_data = pd.concat((train_df.loc[:,'PassengerId':'Embarked'],
                      test_df.loc[:,'PassengerId':'Embarked']))

all_data['Title'] = all_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
categorial.append('Title')
all_data['Cabin'] =  all_data['Cabin'].str[:1]
categorial.append('Cabin')
all_data['Embarked'] = all_data['Embarked'].fillna('S')
all_data.head()

In [ ]:
title_le = LabelEncoder()
for cat in categorial:
    all_data[cat] = title_le.fit_transform(all_data[cat])
all_data.head() 

In [ ]:
all_data.drop('Name', axis=1, inplace=True)
all_data.drop('Ticket', axis=1, inplace=True)
all_data['Parch'] = all_data['Parch'] + all_data['SibSp']
all_data.drop('SibSp', axis=1, inplace=True)
all_data.drop('PassengerId', axis=1, inplace=True)
all_data.drop('Survived', axis=1, inplace=True)
all_data.head()

In [ ]:
all_data["Age"] =all_data["Age"].fillna(all_data.groupby("Title")["Age"].transform("median"))
all_data["Fare"] =all_data['Cabin'].fillna(all_data.groupby("Pclass")["Fare"].transform("median"))
all_data["Cabin"] =all_data['Cabin'].fillna(all_data.groupby("Pclass")["Cabin"].transform("median"))

In [ ]:
all_data.info()

In [ ]:
mms = MinMaxScaler()
cols = ["Cabin", "Age", "Fare", "Title"]
for col in cols:
    all_data[col] = mms.fit_transform(all_data[col].values.reshape(-1, 1))
all_data

In [ ]:
X_train = all_data[:train_df.shape[0]]
X_test = all_data[train_df.shape[0]:]
Y_train = train_df.Survived

X_train.head()
print("\nShape check\n train.shape: ",train_df.shape)
print(" X_Train dim: ",X_train.shape)
print("\n X_test dim: ",X_test.shape)

In [ ]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 13)
scoring = 'accuracy'
score = cross_val_score(clf, X_train, Y_train, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

In [ ]:
models = [KNeighborsClassifier(n_neighbors = 13),DecisionTreeClassifier(),
       RandomForestClassifier(n_estimators=13),GaussianNB(),SVC(),ExtraTreeClassifier(),
      GradientBoostingClassifier(n_estimators=13, learning_rate=1,max_features=3, max_depth =3, random_state = 10),AdaBoostClassifier(n_estimators=12),ExtraTreesClassifier()]
def model_fit():
    scoring = 'accuracy'
    for i in range(len(models)):
        score = cross_val_score(models[i], X_train, Y_train, cv=k_fold, n_jobs=1, scoring=scoring)
        print("Score of Model",i,":",round(np.mean(score)*100,2))
model_fit()

In [ ]:
grid_space={
              'n_estimators':[10,11,12,13,20,25,30,50,100,200],
              'max_features': [1,3,5,7],
              'max_depth': [1,3,5,7],
           }

In [ ]:
model_gradient = GradientBoostingClassifier()
grid = GridSearchCV(model_gradient,param_grid=grid_space,cv=3,scoring='accuracy')
model_grid = grid.fit(X_train,Y_train)
print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))
model_grid.best_params_

In [ ]:
model_gradient = GradientBoostingClassifier(max_depth=model_grid.best_params_['max_depth'], max_features=model_grid.best_params_['max_features'], n_estimators=model_grid.best_params_['n_estimators'])
model_gradient.fit(X_train, Y_train)
prediction = model_gradient.predict(X_test)

In [ ]:
prediction

In [ ]:
submission = pd.DataFrame({"PassengerId":test_df['PassengerId'].values,"Survived":prediction})
submission

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index = False)